# Table of contents

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>Step 1:</b> Set your MURAL OAuth token</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 2:</b> Set your MURAL workspace details</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 3:</b> Get rooms details</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 4:</b> Get murals metadata</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 5:</b> Get widget details for every mural</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 6:</b> Format data into "documents" array</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 7:</b> Save documents array as a project asset</li>
</ol>

## Step 1: Set your MURAL OAuth token

You must get an OAuth token by using OAuth 2.0.  For example, see: [Setting up OAuth 2.0 in Postman](https://developers.mural.co/public/docs/testing-with-postman#setting-up-oauth-20-in-postman)

In [ ]:
g_auth_token = ""

## Step 2: Set your MURAL workspace details

We need the name and ID of the workspace you want to search.

<b>Workspace name</b>

You can copy the name of the workspace right from the MURAL GUI.

<b>workspace ID</b>

You can get the workspace ID from your browser URL.

When you are in the Home page of your workspace, the URL in your browser looks something like this:
```
https://app.mural.co/t/<workspace_id>/home
```

What you need to pass to the MURAL API is just after the `/t/` and before the `/home`.

For example, if you have a workspace with this url:
```
https://app.mural.co/t/teamideas1234/home
```

Then, the workspace ID is: teamideas1234

In [ ]:
g_workspace_name = ""
g_workspace_id = ""

## Step 3: Get rooms details

Use the MURAL API to collect details about any rooms in your workspace.

In [ ]:
import requests
import json
import re

def listRoomsInWorkspace( auth_token, workspace_id ):
    rooms_json = listRoomsInWorkspace_recurse( auth_token, workspace_id, None, [] )
    return formatRoomResults( rooms_json )
    
def listRoomsInWorkspace_recurse( auth_token, workspace_id, next_token, prev_results ):
    # https://developers.mural.co/public/reference/getworkspacerooms
    url = "https://app.mural.co/api/public/v1/workspaces/" + workspace_id + "/rooms?limit=100"
    if next_token is not None:
        url += "&next=" + next_token
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return prev_results
    if ( response_json is None ) or ( "value" not in response_json ) or ( response_json["value"] is None ):
        return prev_results
    prev_results += response_json["value"]
    if ( "next" in response_json ) and ( response_json["next"] is not None ) and ( re.match( r"\S", response_json["next"] ) ):
        #print( url )
        return listRoomsInWorkspace_recurse( auth_token, workspace_id, response_json["next"], prev_results )
    return prev_results

def formatRoomResults( rooms_json ):
    formatted_results = {}
    if rooms_json is not None:
        for room in rooms_json:
            formatted_results[ room["id"] ] = room["name"]
    return formatted_results

In [ ]:
g_rooms = listRoomsInWorkspace( g_auth_token, g_workspace_id )
print( json.dumps( g_rooms, indent=3 ) )

## Step 4: Get murals metadata

Use the MURAL API to get metadata about every mural in the workspace.

In [ ]:
def listMuralsInWorkspace( auth_token, workspace_id ):
    murals_json = listMuralsInWorkspace_recurse( auth_token, workspace_id, None, [] )
    return formatMuralsResults( murals_json )
    
def listMuralsInWorkspace_recurse( auth_token, workspace_id, next_token, prev_results ):
    # https://developers.mural.co/public/reference/getworkspacemurals
    url = "https://app.mural.co/api/public/v1/workspaces/" + workspace_id + "/murals?limit=100"
    if next_token is not None:
        url += "&next=" + next_token
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return prev_results
    if ( response_json is None ) or ( "value" not in response_json ) or ( response_json["value"] is None ):
        return prev_results
    prev_results += response_json["value"]
    if ( "next" in response_json ) and ( response_json["next"] is not None ) and ( re.match( r"\S", response_json["next"] ) ):
        print( url )
        return listMuralsInWorkspace_recurse( auth_token, workspace_id, response_json["next"], prev_results )
    return prev_results

def formatMuralsResults( murals_json ):
    murals_arr = []
    if murals_json is not None:
        for mural in murals_json:
            murals_arr.append( { "id"         : mural["id"],
                                 "title"      : mural["title"],
                                 "room"       : { "id" : mural["roomId"], "name" : "" },
                                 "creator"    : mural["createdBy"]["email"],
                                 "created"    : mural["createdOn"],
                                 "text_arr"   : [],
                                 "sticky_arr" : [],
                                 "shape_arr"  : [],
                                 "link"       : mural["visitorsSettings"]["link"],
                                 "thumbnail"  : mural["thumbnailUrl"],
                                 "workspace_name" : g_workspace_name } )
    return murals_arr

In [ ]:
g_murals = listMuralsInWorkspace( g_auth_token, g_workspace_id )
print( json.dumps( g_murals[0], indent=3 ) )

## Step 5: Get widget details for every mural

Use the MURAL API to get widget details for every mural in the workspace.

In [ ]:
def listWidgets( auth_token, mural_id ):
    widgets_json = listWidgets_recurse( auth_token, mural_id, None, [] )
    return widgets_json, formatWidgetsResults( widgets_json )

def listWidgets_recurse( auth_token, mural_id, next_token, prev_results ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    if next_token is not None:
        url += "&next=" + next_token
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return prev_results
    if ( response_json is None ) or ( "value" not in response_json ) or ( response_json["value"] is None ):
        return prev_results
    prev_results += response_json["value"]
    if ( "next" in response_json ) and ( response_json["next"] is not None ) and ( re.match( r"\S", response_json["next"] ) ):
        print( url )
        return listWidgets_recurse( auth_token, mural_id, response_json["next"], prev_results )
    return prev_results

def formatWidgetsResults( widgets_json ):
    widgets_final = { "text_arr" : [], "sticky_arr" : [], "shape_arr" : [] }
    if widgets_json is not None:
        for widget in widgets_json:
            widget_id = widget["id"]
            widget_json = { "id" : widget_id, 
                            "text" : "__SPLTB__" + widget_id + "|" + re.sub( r"\<[^>]*?\>", "", cleanText( widget["text"] ) ) + "__SPLTE__",
                            "backgroundColor" : widget["style"]["backgroundColor"]
                          }
            if "sticky note" == widget["type"]:
                widget_json["shape"] = widget["shape"]
                widgets_final["sticky_arr"].append( widget_json )
            elif "shape" == widget["type"]:
                widget_json["shape"] = widget["shape"]
                widgets_final["shape_arr"].append( widget_json )
            elif "text" == widget["type"]:
                widgets_final["text_arr"].append( widget_json )
    return widgets_final

def cleanText( txt_in ):
    txt_out = re.sub( r"\<[^\>]+?\>", "", txt_in )
    txt_out = re.sub( r"\s+", " ", txt_out )
    txt_out = re.sub( r"^\s+", "", txt_out )
    txt_out = re.sub( r"\s+$", "", txt_out )
    return txt_out

In [ ]:
# Test the function
widgets_raw, widgets_formatted = listWidgets( g_auth_token, g_murals[0]["id"] )
print( json.dumps( widgets_formatted, indent=3 ) )

## Step 6: Format data into "documents" array

Each mural will be one _document_ (JSON format) in Watson Discovery.

In [ ]:
def createDocumentsArr( auth_token, murals_arr, rooms_json ):
    documents_arr = []
    for mural in murals_arr:
        for room_id in rooms_json.keys():
            if room_id == mural["room"]["id"]:
                mural["room"]["name"] = rooms_json[ room_id ]
        widgets_raw, widgets_formatted = listWidgets( auth_token, mural["id"] )
        mural["text_arr"]   = widgets_formatted["text_arr"]
        mural["sticky_arr"] = widgets_formatted["sticky_arr"]
        mural["shape_arr"]  = widgets_formatted["shape_arr"]
        documents_arr.append( mural )
    return documents_arr

In [ ]:
g_documents_arr = createDocumentsArr( g_auth_token, g_murals, g_rooms )
print( json.dumps( g_documents_arr, indent=3 ) )

## Step 7: Save documents array as a project asset

Save the documents array as a JSON file in the Watson Studio project assets.

To be able to easily save files as assets in the project, you need to create a project token and add it to your notebook.

Follow the steps in the section titled "Use the library" in this topic: [Using project-lib for Python](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html?context=cpdaas&audience=wdp)

**The project token is added in the very first cell at the top of the notebook. Don't forget to scroll up and run that cell.**

(If you forget to run the inserted cell, you'll see the error name 'project' is not defined when you try to run the next cell below.)

In [ ]:
project.save_data( "documents_arr.json", json.dumps( g_documents_arr, indent=3 ), set_project_asset=True, overwrite=True )